# Making Dataset For GRU-ODE-Bayes (by fujimori)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import datetime
from datetime import timedelta
import numpy as np

from tqdm import tqdm
import time

file_path="D:/mimic_iii/csv/"
outfile_path="D:/mimic_iii/clean_data/"

In [2]:
complete_tensor=pd.read_csv(outfile_path+"complete_tensor.csv") #Full data

FileNotFoundError: [Errno 2] No such file or directory: 'D:/mimic_iii/csv/clean_data/complete_tensor.csv'

In [8]:
if "Unnamed: 0" in complete_tensor.columns:
    complete_tensor=complete_tensor.drop("Unnamed: 0",axis=1)
complete_tensor.head()

,UNIQUE_ID,LABEL_CODE,TIME_STAMP,VALUENUM,MEAN,STD,VALUENORM
0,16990,0,0.0,15.0,14.331770,3.957040,0.168871
1,16990,0,26.0,12.0,14.331770,3.957040,-0.589271
2,16990,0,59.0,10.0,14.331770,3.957040,-1.094699
3,16990,1,0.0,25.0,23.685408,4.549916,0.288927
4,16990,1,26.0,25.0,23.685408,4.549916,0.288927


In [9]:
complete_tensor=complete_tensor.sort_values(["UNIQUE_ID","TIME_STAMP"])

In [10]:
complete_tensor

,UNIQUE_ID,LABEL_CODE,TIME_STAMP,VALUENUM,MEAN,STD,VALUENORM
2792641,0,66,0.0,100.0,108.243422,30.350528,-0.271607
2792642,0,84,0.0,40.0,42.878235,11.025300,-0.261057
2792643,0,65,0.0,20.0,16.361990,6.020163,0.604304
2792647,0,67,0.0,81.0,211.578203,150.612628,-0.866980
2792499,0,0,36.0,12.0,14.331770,3.957040,-0.589271
...,...,...,...,...,...,...,...
394357,21245,62,84.0,45.0,114.996140,122.726504,-0.570342
394344,21245,57,90.0,10.0,14.899846,18.546874,-0.264187
394349,21245,56,90.0,100.0,49.904313,33.239474,1.507114
394358,21245,62,90.0,80.0,114.996140,122.726504,-0.285156


## 記録が1つでもある時刻に対して、96変数についてマスクの作成含め処理をしていく

In [11]:
# 患者毎に記録がある時刻の総数
bin_sum=0
for id in tqdm(list(complete_tensor["UNIQUE_ID"].unique())):
    bin_sum+=complete_tensor[complete_tensor["UNIQUE_ID"]==id]["TIME_STAMP"].nunique()


100%|███████████████████████████████████████████████████████████████████████████| 21246/21246 [01:38<00:00, 215.28it/s]


In [12]:
bin_sum

552499

In [ ]:
# bin_sum=1043381

In [13]:
# (ID,TIME)+{val0-95,mask0-95}を記録するnumpy配列を作成
val_array=np.zeros((bin_sum,2+96))
mask_array=np.zeros((bin_sum,2+96))
# pd.DataFrame(val_array)

In [14]:
import random
random.seed(0)

In [15]:
# a=[1,2,3,4,5]
# for i in random.sample(a,len(a)):
#     print(i)

In [16]:
# UNIQUE_IDの集合を得る,これをrandom.sampleでシャッフルしてから編集する
# randomはlist対象なのでndarray->listにする
groups=list(complete_tensor["UNIQUE_ID"].unique())

In [17]:
# UNIUE_ID毎に1時刻ずつ記録してある変数を横に展開していく、マスクも記録
# 60binだと300万行あるので気長に(10^7)
# 

spenttime=[] # 実行時間記録
row_idx=0
sample_id=0

# テスト用
# sampling=random.sample(groups,10)

# 全データ用
sampling=random.sample(groups,len(groups))

for unique_id in tqdm(sampling):
    start=time.time()
    time_stamps=complete_tensor[complete_tensor["UNIQUE_ID"]==unique_id]["TIME_STAMP"].unique()
    for time_stamp in time_stamps:
        sample_time_df=complete_tensor[(complete_tensor["UNIQUE_ID"]==unique_id) & (complete_tensor["TIME_STAMP"]==time_stamp)]
        val_array[row_idx,0]=sample_id
        val_array[row_idx,1]=time_stamp
        mask_array[row_idx,0]=sample_id
        mask_array[row_idx,1]=time_stamp
        for code,valnum in zip(sample_time_df["LABEL_CODE"],sample_time_df["VALUENORM"]):
            val_array[row_idx,code+2]=valnum
            mask_array[row_idx,code+2]=1
        row_idx+=1
    sample_id+=1
    elapsed_time = time.time() - start
    spenttime.append(elapsed_time)

100%|██████████████████████████████████████████████████████████████████████████| 21246/21246 [1:31:42<00:00,  3.86it/s]


In [ ]:
plt.hist(spenttime)

In [ ]:
# sampling=[3,7,1,2,5,4,6]
sample_unique_id_dict=dict(zip(range(len(sampling)),sampling))
sample_unique_id_dict

In [19]:
import pickle

def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data

In [20]:
# UNIQUE_IDとsample_idの接続用dictionaryの保存
# ダンプ
pickle_dump(sample_unique_id_dict, outfile_path+'sample_unique_id_dict.pickle')

In [ ]:
# ロード
print(pickle_load( outfile_path+'sample_unique_id_dict.pickle')) 

In [22]:
np.save(outfile_path+'val_array',val_array)
np.save(outfile_path+'mask_array',mask_array)

In [ ]:
# np.savetxt(outfile_path+'val_array.csv', val_array[0:1000,:], delimiter=',' ,)

In [ ]:
# np.array(spenttime).mean()*23000/3600

In [ ]:
# val_array.shape

In [23]:
val_columns=["ID","Time"]+["Value_"+str(i) for i in range(96)]
mask_columns=["ID","Time"]+["Mask_"+str(i) for i in range(96)]

In [24]:
print(val_columns[0:3])
print(mask_columns[0:3])

['ID', 'Time', 'Value_0']
['ID', 'Time', 'Mask_0']


In [25]:
val_df=pd.DataFrame(data=val_array,columns=val_columns,dtype='float')
val_df.head()

,ID,Time,Value_0,Value_1,Value_2,Value_3,Value_4,Value_5,Value_6,Value_7,...,Value_86,Value_87,Value_88,Value_89,Value_90,Value_91,Value_92,Value_93,Value_94,Value_95
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,14.0,0.0,0.0,0.0,0.0,0.0,-0.473959,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,16.0,0.0,0.0,0.0,0.0,0.0,-0.506722,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,19.0,0.0,0.0,0.0,0.0,0.0,-0.244619,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.006564,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
mask_df=pd.DataFrame(data=mask_array,columns=mask_columns,dtype='float')
mask_df.head()

,ID,Time,Mask_0,Mask_1,Mask_2,Mask_3,Mask_4,Mask_5,Mask_6,Mask_7,...,Mask_86,Mask_87,Mask_88,Mask_89,Mask_90,Mask_91,Mask_92,Mask_93,Mask_94,Mask_95
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,14.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,16.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,19.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,21.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
res_df=pd.merge(val_df,mask_df)

In [28]:
res_df.to_csv(outfile_path+'GRU_ODE_DataFrame.csv')

In [29]:
res_df.tail()

,ID,Time,Value_0,Value_1,Value_2,Value_3,Value_4,Value_5,Value_6,Value_7,...,Mask_86,Mask_87,Mask_88,Mask_89,Mask_90,Mask_91,Mask_92,Mask_93,Mask_94,Mask_95
552494,21245.0,88.0,-0.083843,1.168064,0.6396,0.752594,0.171985,-0.353828,0.573695,0.284164,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
552495,21245.0,90.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
552496,21245.0,92.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
552497,21245.0,94.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
552498,21245.0,95.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
res_df.to_csv(outfile_path+'GRU_ODE_Dataset.csv',index=False)